In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
MODEL_NAME = "gpt-4o-mini"

## Tools

### PDF Retriever and Retrieval Chain

In [3]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 로드합니다.
pdf_chain_obj = PDFRetrievalChain(["data/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

# retriever와 chain을 생성합니다.
pdf_retriever = pdf_chain_obj.retriever
pdf_chain = pdf_chain_obj.chain

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


data/SPRI_AI_Brief_2023년12월호_F.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


### Query Router (Vector store or Web Search)

#### Route Prompt

In [4]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

system_message = """You are an expert at routing a user question to a vectorstore or web search.
                    The vectorstore contains documents related to DEC 2023 AI Brief Report(SPRI) with Samsung Gause, Anthropic, etc.
                    Use the vectorstore for questions on these topics. Otherwise, use web-search."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "{query}"),
    ]
)

#### Query Router

In [5]:
class RouteQuery(BaseModel):
    """ Route a user query to the most relevant datasource """

    datasource: Literal["vector_store", "web_search"] = Field(
        ..., # required
        description= "Route the user query to either the vectorstore or the web search engine based on its content."
    )

llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery) # Convert the result to RouteQuery type.

query_router = route_prompt | structured_llm_router

#### Test Query Router

In [6]:
print(
    query_router.invoke(
        {"query": "AI Brief 에서 삼성전자가 만든 생성형 AI 의 이름은?"}
    )
)

datasource='vector_store'


In [7]:
print(
    query_router.invoke(
        {"query": "판교에서 가장 맛있는 딤섬집 찾아줘"}
    )
)

datasource='web_search'


### Retrieval Grader

#### System Prompt

In [8]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

system = """ You are a grader evaluating the relevance of a retrieved document to a user query.
            If the document contains keywords or semantic meanings related to the query, consider it relevant.
            This is not a strict test — the goal is to filter out irrelevant results clearly.
            Return a binary score: "yes" if the document is relevant, or "no" if it is not."""

retrieval_grader_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {documents} \n\n User query: {query}"),
    ]
)

#### Retrieval Grader

In [9]:
class GradeDocuments(BaseModel):
    binary_score: str = Field(
        description="Are the documents relevant to the user query? Answer 'yes' or 'no'."
    )

llm = ChatOpenAI(model=MODEL_NAME, temperature = 0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

retrieval_grader = retrieval_grader_prompt | structured_llm_grader

#### Test Retrieval Grader

In [10]:
query = "삼성전자가 만든 생성형 AI 의 이름은?"
retrieved_docs = pdf_retriever.invoke(query)
retrieved_doc = retrieved_docs[1].page_content
print(retrieval_grader.invoke({"query":query, "documents":retrieved_doc}))

binary_score='yes'


In [11]:
filtered_docs = []
for doc in retrieved_docs:
    result = retrieval_grader.invoke(
        {
            "query": query,
            "documents": doc.page_content,
        }
    )
    if result.binary_score == "yes":
        filtered_docs.append(doc)

In [12]:
print(filtered_docs)

[Document(id='61e5bff7-72c2-4199-a330-cbc69b255c28', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'), Document(id='83962b39-ff5f-494f-bd3e-ce8ca4acca0b', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate'

### RAG Chain for answer generation

In [30]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 지침 포함한 프롬프트 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system.

Your job is to answer the user query using only the provided context.

- Be concise and accurate.
- If the answer is not in the context, say "I don't know".
- List the source(s) if available, otherwise omit.

User Query: {query}

Provided Context:
{context}

Answer:""")
])

In [32]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

query = "삼성전자가 만든 생성형 AI 의 이름은?"

# prompt = hub.pull("qa-rag-prompt/general")

llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0.9)

def format_docs(retrieved_docs):
    return "\n\n".join(
        [
            f"""<document>
                    <content>
                        {doc.page_content}
                    </content>
                    <source>
                        {doc.metadata["source"]}
                    </source>
                    <page>
                        {doc.metadata["page"]+1}
                    </page>
                </docuemnt>""" for doc in retrieved_docs
        ]
    )

def format_docs(docs):
    return "\n\n".join([f"{doc.page_content} (Source: {doc.metadata['source']}, Page {doc.metadata['page']+1})" for doc in docs])


rag_chain = prompt | llm | StrOutputParser()

In [33]:
answer = rag_chain.invoke({"context": format_docs(retrieved_docs), "query": query})
print(answer)

삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다.


In [24]:
print(prompt)
print(llm)
print(StrOutputParser())
print(rag_chain)
print(answer)
rag_chain

input_variables=['context', 'query'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'qa-rag-prompt', 'lc_hub_repo': 'general', 'lc_hub_commit_hash': 'ed61f794b1d06a1dbf28e4e20f4e7f59234aa58d65ddca2ddae4d73adb3e3dd9'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. \\n\\n\nYour primary mission is to answer queries based on the provided context or chat history. \\n\\n\nEnsure your response is concise and directly addresses the query without any additional narration. \\n\\n\n\nInstructions: \\n\\n \nCarefully read and understand the provided context. \\n\\n\n\nIdentify the key information relevant to the query. \\n\\n\n\nFormulate a concise answer based on the relevant information. \\n\\n\n\nEnsure your final answer directly addresses the query. \\n\\n\n\n

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'qa-rag-prompt', 'lc_hub_repo': 'general', 'lc_hub_commit_hash': 'ed61f794b1d06a1dbf28e4e20f4e7f59234aa58d65ddca2ddae4d73adb3e3dd9'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. \\n\\n\nYour primary mission is to answer queries based on the provided context or chat history. \\n\\n\nEnsure your response is concise and directly addresses the query without any additional narration. \\n\\n\n\nInstructions: \\n\\n \nCarefully read and understand the provided context. \\n\\n\n\nIdentify the key information relevant to the query. \\n\\n\n\nFormulate a concise answer based on the relevant information. \\n\\n\n\nEnsure your final answer directly addresse

In [25]:
print(prompt)

input_variables=['context', 'query'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'qa-rag-prompt', 'lc_hub_repo': 'general', 'lc_hub_commit_hash': 'ed61f794b1d06a1dbf28e4e20f4e7f59234aa58d65ddca2ddae4d73adb3e3dd9'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. \\n\\n\nYour primary mission is to answer queries based on the provided context or chat history. \\n\\n\nEnsure your response is concise and directly addresses the query without any additional narration. \\n\\n\n\nInstructions: \\n\\n \nCarefully read and understand the provided context. \\n\\n\n\nIdentify the key information relevant to the query. \\n\\n\n\nFormulate a concise answer based on the relevant information. \\n\\n\n\nEnsure your final answer directly addresses the query. \\n\\n\n\n